In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Model, self).__init__()

        # ONLY TRU IN CASE OF DUCKIETOWN:
        flat_size = 32 * 9 * 14

        self.lr = nn.LeakyReLU()
        self.tanh = nn.Tanh()
        self.sigm = nn.Sigmoid()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv3 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv4 = nn.Conv2d(32, 32, 4, stride=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.5)

        self.lin1 = nn.Linear(flat_size, 512)
        self.lin2 = nn.Linear(512, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = self.bn1(self.lr(self.conv1(x)))
        x = self.bn2(self.lr(self.conv2(x)))
        x = self.bn3(self.lr(self.conv3(x)))
        x = self.bn4(self.lr(self.conv4(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = self.lr(self.lin1(x))

        # because we don't want our duckie to go backwards
        x = self.lin2(x)
        x[:, 0] = self.max_action * self.sigm(x[:, 0])  # because we don't want the duckie to go backwards
        x[:, 1] = self.tanh(x[:, 1])

        return x

In [6]:
import time
import random
import argparse
import math
import json
from functools import reduce
import operator

import numpy as np
import torch
import torch.optim as optim

from learning.utils.env import launch_env
from learning.utils.wrappers import NormalizeWrapper, ImgWrapper, \
    DtRewardWrapper, ActionWrapper, ResizeWrapper
from learning.utils.teacher import PurePursuitExpert

from gail.models import *
from torch.utils.tensorboard import SummaryWriter
# from learning.imitation.pytorch.model import Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def _train(args):
    
    writer = SummaryWriter()
    
    env = launch_env()
    env = ResizeWrapper(env)
    env = NormalizeWrapper(env) 
    env = ImgWrapper(env)
    env = ActionWrapper(env)
    env = DtRewardWrapper(env)
    print("Initialized Wrappers")

    observation_shape = (None, ) + env.observation_space.shape
    action_shape = (None, ) + env.action_space.shape

    # Create an imperfect demonstrator
    expert = PurePursuitExpert(env=env)

    observations = []
    actions = []
    
    if args.get_samples:
    # let's collect our samples
        for episode in range(0, args.episodes):
            print("Starting episode", episode)
            for steps in range(0, args.steps):
                # use our 'expert' to predict the next action.
                action = expert.predict(None)
                observation, reward, done, info = env.step(action)
                observations.append(observation)
                actions.append(action)
            env.reset()
        env.close()
        
        actions = np.array(actions)
        observations = np.array(observations)
        torch.save(actions, 'data_a.pt')
        torch.save(observations, 'data_o.pt')

    else:
        actions = torch.load('data_a.pt')
        observations = torch.load('data_o.pt')


    
    model = Generator(action_dim=2)
    model.train().to(device)

    # weight_decay is L2 regularization, helps avoid overfitting
    optimizer = optim.Adam(
        model.parameters(),
        lr=0.004,
        weight_decay=1e-3
    )
    
    avg_loss = 0
    for epoch in range(args.epochs):
        optimizer.zero_grad()

        batch_indices = np.random.randint(0, observations.shape[0], (args.batch_size))
        obs_batch = torch.from_numpy(observations[batch_indices]).float().to(device)
        act_batch = torch.from_numpy(actions[batch_indices]).float().to(device)

        model_actions = model(obs_batch)
        
        loss = (model_actions - act_batch).norm(2).mean()
        loss.backward()
        optimizer.step()
        
        loss = loss.data.item()
        avg_loss = avg_loss * 0.995 + loss * 0.005

#         print('epoch %d, loss=%.3f' % (epoch, avg_loss))
        
        writer.add_scalar('Loss/train', avg_loss, epoch)
#         writer.add_scalar('Loss/test', np.random.random(), n_iter)
#         writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
#         writer.add_scalar('Accuracy/test', np.random.random(), n_iter)
        # Periodically save the trained model
        if epoch % 200 == 0:
            torch.save(model.state_dict(), 'test.pt')
            print('epoch %d, loss=%.3f' % (epoch, avg_loss))


In [1]:
from gail.train import _train

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        
args={'episodes':3, 
      'seed':1234, 
      'steps':50, 
      'batch_size':32,
      'epochs':2000,
      'model_directory':'models/',
      'get_samples':False}
args = Struct(**args)

_train(args)

INFO:gym-duckietown:gym-duckietown 2019.0.0

INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
C:\Users\micha\miniconda3\envs\gym-duckietown\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(co

DEBUG:gym-duckietown:f_pos: [4.11050548 0.         2.28364106]
DEBUG:gym-duckietown:[3.50186974 0.         2.10261411] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [3.5769519  0.         2.04041348]
DEBUG:gym-duckietown:l_pos: [3.50186974 0.         2.10261411]
DEBUG:gym-duckietown:r_pos: [3.65203407 0.         1.97821284]
DEBUG:gym-duckietown:f_pos: [3.50231114 0.         1.95031488]
DEBUG:gym-duckietown:[3.47260658 0.         1.77460142] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-d

Initialized Wrappers
epoch 0, loss=0.063
epoch 200, loss=27.764
epoch 400, loss=11.000
epoch 600, loss=4.733
epoch 800, loss=2.409
epoch 1000, loss=1.595
epoch 1200, loss=1.386
epoch 1400, loss=1.213
epoch 1600, loss=1.169
epoch 1800, loss=1.129


In [2]:
from learning.utils.env import launch_env
from learning.utils.wrappers import NormalizeWrapper, ImgWrapper, \
    DtRewardWrapper, ActionWrapper, ResizeWrapper
from learning.utils.teacher import PurePursuitExpert
from gail.models import *

env = launch_env()
env = ResizeWrapper(env)
env = NormalizeWrapper(env) 
env = ImgWrapper(env)
env = ActionWrapper(env)
env = DtRewardWrapper(env)
print("Initialized Wrappers")


observation_shape = (None, ) + env.observation_space.shape
action_shape = (None, ) + env.action_space.shape

model = Generator(action_dim=2)
model.load_state_dict(torch.load('test.pt'))
model.eval()
# model.to(device)
# Create an imperfect demonstrator

observations = []
actions = []

# let's collect our samples
for episode in range(0, args.episodes):
    print("Starting episode", episode)
    observation = env.reset()
    for steps in range(0, args.steps):
        # use our 'expert' to predict the next action.
        observation = torch.tensor(observation, device="cpu").float()
        action = model.forward(observation.unsqueeze(0)).detach().numpy()[0]
        print(action)
        observation, reward, done, info = env.step(action)
        observations.append(observation)
        actions.append(action)
        env.render()
    env.reset()
env.close()


DEBUG:gym-duckietown:loading map file "C:\Users\micha\Documents\Projects\gym_things\gym-duckietown\gym_duckietown\maps\loop_empty.yaml"
DEBUG:gym-duckietown:[4.13719561 0.         1.88880899] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [4.0873995 0.        1.9726338]
DEBUG:gym-duckietown:l_pos: [4.03760339 0.         2.0564586 ]
DEBUG:gym-duckietown:r_pos: [4.13719561 0.         1.88880899]
DEBUG:gym-duckietown:f_pos: [3.98680973 0.         1.91287847]
DEBUG:gym-duckietown:[4.10908822 0.         2.30642426] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture

DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [4.02300463 0.         1.79236447]
DEBUG:gym-duckietown:l_pos: [3.94666773 0.         1.7317103 ]
DEBUG:gym-duckietown:r_pos: [4.09934153 0.         1.85301864]
DEBUG:gym-duckietown:f_pos: [4.09578963 0.         1.70076019]
DEBUG:gym-duckietown:[4.17105167 0.         1.80077003] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [4.09036721 0.         1.85550821]
DEBUG:gym-duckietown:l_pos: [4.00968276 0.         1.9102464 ]
DEBUG:gym-duckietown:r_pos: [4.17105167 0.         1.80077003]
DEBUG:gym-duckietown:f_pos: [4.02468139 0.         1.75868687]
DEBUG:gym-duckietown:[3.49620504 0.         2.20742184] corres

Initialized Wrappers


DEBUG:gym-duckietown:[2.33703727 0.         2.57700384] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.33703727 0.         2.57700384]
DEBUG:gym-duckietown:l_pos: [2.24568806 0.         2.542922  ]
DEBUG:gym-duckietown:r_pos: [2.42838647 0.         2.61108568]
DEBUG:gym-duckietown:f_pos: [2.37793548 0.         2.46738479]
DEBUG:gym-duckietown:[2.28873639 0.         2.68406941] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: 

Starting episode 0


...

INFO:gym-duckietown:Pos: [1.68880057 0.         0.93992464] angle 8.344048074961732
INFO:gym-duckietown:Pos: [1.68880057 0.         0.93992464] angle 8.344048074961732



[0.77460915 1.6080488 ]


INFO:gym-duckietown:Pos: [1.67073365 0.         0.9093884 ] angle 8.432436640280768
INFO:gym-duckietown:Pos: [1.67073365 0.         0.9093884 ] angle 8.432436640280768
DEBUG:gym-duckietown:[2.70689661 0.         0.52127018] corresponds to tile at (4, 0) which is not drivable: {'coords': (4, 0), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.72096709 0.         0.61774956]
DEBUG:gym-duckietown:l_pos: [2.73503756 0.         0.71422894]
DEBUG:gym-duckietown:r_pos: [2.70689661 0.         0.52127018]
DEBUG:gym-duckietown:f_pos: [2.60519182 0.         0.63463413]
DEBUG:gym-duckietown:[2.89188152 0.         1.19337946] corresponds to tile at (4, 2) which is not drivable: {'coords': (4, 2), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture

DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.79630767 0.         0.61665626]
DEBUG:gym-duckietown:l_pos: [2.70656034 0.         0.654757  ]
DEBUG:gym-duckietown:r_pos: [2.88605501 0.         0.57855553]
DEBUG:gym-duckietown:f_pos: [2.75058679 0.         0.50895946]
DEBUG:gym-duckietown:[2.35921308 0.         0.56671295] corresponds to tile at (4, 0) which is not drivable: {'coords': (4, 0), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.40602588 0.         0.65223963]
DEBUG:gym-duckietown:l_pos: [2.45283867 0.         0.7377663 ]
DEBUG:gym-duckietown:r_pos: [2.35921308 0.         0.56671295]
DEBUG:gym-duckietown:f_pos: [2.30339387 0.   

DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.55826064 0.         0.64954754]
DEBUG:gym-duckietown:l_pos: [2.46078006 0.         0.65149372]
DEBUG:gym-duckietown:r_pos: [2.65574121 0.         0.64760137]
DEBUG:gym-duckietown:f_pos: [2.55592523 0.         0.53257085]
DEBUG:gym-duckietown:[2.86842933 0.         1.24497133] corresponds to tile at (4, 2) which is not drivable: {'coords': (4, 2), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x00000272959456A0>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.82562443 0.         1.15737001]
DEBUG:gym-duckietown:l_pos: [2.78281952 0.         1.0697687 ]
DEBUG:gym-duckietown:r_pos: [2.86842933 0.         1.24497133]
DEBUG:gym-duckietown:f_pos: [2.930746   0.   

In [4]:
import torch
actions = torch.load('data_a.pt')
observations = torch.load('data_o.pt')

actions.max()

5.702396438546007